In [1]:
import os
os.chdir("../")
%pwd

'/Users/tapankheni/Data_Science/Data Science Projects/Wafers_Fault_Prediction'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    x_test_data_path: Path
    y_test_data_path: Path
    model_path: Path
    metrics_name: str

In [3]:
from WafersFault.constants import PARAMS_YAML_FILE_PATH, CONFIG_YAML_FILE_PATH, SCHEMA_YAML_FILE_PATH
from WafersFault.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_YAML_FILE_PATH,
        params_filepath = PARAMS_YAML_FILE_PATH,
        schema_filepath = SCHEMA_YAML_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            x_test_data_path=config.x_test_data_path,
            y_test_data_path=config.y_test_data_path,
            model_path=config.model_path,
            metrics_name=config.metrics_name
        )

        return model_evaluation_config

In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import joblib
from WafersFault import logger

In [17]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig) -> None:
        self.config = config

    def eval_matrix(self, y_true, y_pred):
        precision = precision_score(y_true=y_true, y_pred=y_pred)
        recall = recall_score(y_true=y_true, y_pred=y_pred)
        f1 = f1_score(y_true=y_true, y_pred=y_pred)
        accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
        return precision, recall, f1, accuracy
    
    def save_matrix(self):
        X_test = np.load(self.config.x_test_data_path)
        y_test = np.load(self.config.y_test_data_path)

        model = joblib.load(self.config.model_path)

        y_pred = model.predict(X_test)

        (precision, recall, f1, accuracy) = self.eval_matrix(y_pred=y_pred, y_true=y_test)

        if accuracy < 0.5:
            raise Exception("no best model found with an accuracy more than the threshold 0.6.")
        
        logger.info("best model found on both training and testing data")
        
        scores = {
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "accuracy_score": accuracy
        }

        save_json(path=Path(os.path.join(self.config.root_dir, self.config.metrics_name)), data=scores)

In [18]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    obj = ModelEvaluation(config=model_evaluation_config)
    obj.save_matrix()

except Exception as e:
    raise e

[2024-03-12 13:04:41,416: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-12 13:04:41,418: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-12 13:04:41,456: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-12 13:04:41,460: INFO: common: created directory at: artifacts]
[2024-03-12 13:04:41,461: INFO: common: created directory at: artifacts/model_evaluation]
[2024-03-12 13:04:41,477: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
